In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

### ---->Skewness

In [0]:
import random
df = spark.createDataFrame([random.randint(1,100) for i in range(100000) ], schema=['col1'] ).withColumn('col2', when(col('col1')>9, lit('country_1')).when( (col('col1')<9) & (col('col1')>5), lit('country_2') ).otherwise('country_3')  )

df.display()

In [0]:
df.groupBy('col2').count().orderBy('col2').display()

In [0]:
import random
df_dim = spark.createDataFrame(['country_1','country_2','country_3'], schema=['col1'])

df_dim.display()

In [0]:
df_join = df.join(df_dim, on=[df.col2==df_dim.col1], how='inner')
df_join.display()

In [0]:
df_join.withColumn('partition_nm', spark_partition_id() ).groupBy('partition_nm').count().display() 

### handling Skewness
# 1.salting
it is usefull when one table contains similar king of data

In [0]:
num_buckets =10
salted_large_df = df.withColumn(\
    'salt',(rand() = num_buckets).cast(IntegerType())
).withColumn(
    'salted_country_id',concat(col('col2'), lit('_'),col('salt'))
)

salted_large_df.groupBy(salted_country_id).count().display()

### 2.broadcast join
if the size of small table is 10mb then spark bydefault go for broadcast join\
if the size of small table is more than 10mb then spark bydefault go for sortmerge join\
but if the size of small table is more than 10mb and we want to perferm broadcast join then we forcefully can do\
it is usefull when two different table,which one contains huge numbers of record and second oee contains less record

In [0]:
df.join(broadcast(df_dim), on = df.col2==df_dim.col1, how = 'inner').display()

### ---->z-ordering

In [0]:
from delta.tables import DeltaTable

# Assume 'path_to_table' is the path to your Delta table
delta_table = DeltaTable.forName(spark, 'emp')

# Z-Order the table by 'col2'
delta_table.optimize().executeZOrderBy("deptno")